In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split


In [2]:
def make_dataset(path):
    images=[]
    angry=0
    sad=0
    happy=0
    other=0
    for dir in os.listdir(path):
        file=os.path.join(path,dir)
        for filename in os.listdir(file):
            new_path=os.path.join(file,filename)
            for img in os.listdir(new_path):
                image=cv2.imread(os.path.join(new_path,img))
                image=cv2.resize(image,(224,224))
                if image is not None:
                    images.append(image)
                    if filename=="Angry":
                        angry+=1
                    elif filename=="Sad":
                        sad+=1
                    elif filename=="happy":
                        happy+=1
                    else:
                        other+=1
    Angry=np.zeros(angry)
    Other=np.ones(other)
    Sad=np.repeat(2,sad)
    Happy=np.repeat(3,happy)
    label=np.hstack((Angry,Other,Sad,Happy))
    label=tf.keras.utils.to_categorical(label,num_classes=4)
    return images,label

In [3]:
images,label=make_dataset("Master Folder")
images=np.array(images)
print(images.shape,label.shape)

(1074, 224, 224, 3) (1074, 4)


In [4]:
train_X,trail_X,train_label,trail_label=train_test_split(images,label,test_size=0.3,random_state=42,shuffle=True,stratify=label)
validation_X,test_X,validation_label,test_label=train_test_split(images,label,test_size=0.5,random_state=42,shuffle=True,stratify=label)

In [5]:
def preprocess_image(images):
    image=tf.cast(images,tf.float32)/255.0
    return image

In [6]:
train_X=preprocess_image(train_X)
validation_X=preprocess_image(validation_X)
test_X=preprocess_image(test_X)

In [50]:
callbacks=[EarlyStopping(monitor = "val_loss", patience = 10, 
            restore_best_weights = True, mode='min')
            ,ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, mode='min')
            ,ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
            ]

In [63]:
def model_build(inputs):
    
    x=tf.keras.layers.Conv2D(16,activation='relu',kernel_size=3,input_shape=(224,224,3))(inputs)
    x=tf.keras.layers.AveragePooling2D((2,2))(x)
    
    x=tf.keras.layers.Conv2D(32,activation='relu',kernel_size=3)(x)
    x=tf.keras.layers.AveragePooling2D((2,2))(x)
    
    x=tf.keras.layers.Conv2D(64,activation='relu',kernel_size=3)(x)
    x=tf.keras.layers.AveragePooling2D((13,13))(x)
    
    x=tf.keras.layers.Flatten()(x)
    x=tf.keras.layers.Dropout(0.5)(x)
    x=tf.keras.layers.Dense(256,activation='relu')(x)
    x=tf.keras.layers.Dropout(0.5)(x)
    x=tf.keras.layers.Dense(4,activation='sigmoid')(x)
    
    return x

In [64]:
def define_compile_model():
    inputs=tf.keras.layers.Input(shape=(224,224,3))
    output=model_build(inputs)
    model=tf.keras.Model(inputs=inputs,outputs=output)
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [65]:
model=define_compile_model()
model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_36 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 average_pooling2d_36 (Avera  (None, 111, 111, 16)     0         
 gePooling2D)                                                    
                                                                 
 conv2d_37 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 average_pooling2d_37 (Avera  (None, 54, 54, 32)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_38 (Conv2D)          (None, 52, 52, 64)        184

In [68]:
epochs=30
batch_size=64
history = model.fit(train_X, train_label,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(validation_X, validation_label),callbacks=callbacks)


Epoch 1/30
12/12 [==============================] - 17s 1s/step - loss: 0.3669 - accuracy: 0.8575 - val_loss: 0.3006 - val_accuracy: 0.9534 - lr: 0.0010
Epoch 2/30
12/12 [==============================] - 17s 1s/step - loss: 0.4345 - accuracy: 0.8309 - val_loss: 0.2880 - val_accuracy: 0.9460 - lr: 0.0010
Epoch 3/30
12/12 [==============================] - 16s 1s/step - loss: 0.4302 - accuracy: 0.8216 - val_loss: 0.3079 - val_accuracy: 0.9572 - lr: 0.0010
Epoch 4/30
12/12 [==============================] - 16s 1s/step - loss: 0.4176 - accuracy: 0.8522 - val_loss: 0.2837 - val_accuracy: 0.9572 - lr: 0.0010
Epoch 5/30
12/12 [==============================] - 16s 1s/step - loss: 0.3885 - accuracy: 0.8482 - val_loss: 0.2980 - val_accuracy: 0.9628 - lr: 0.0010
Epoch 6/30
12/12 [==============================] - 15s 1s/step - loss: 0.3917 - accuracy: 0.8482 - val_loss: 0.2719 - val_accuracy: 0.9683 - lr: 0.0010
Epoch 7/30
12/12 [==============================] - 15s 1s/step - loss: 0.4052 - a

In [69]:
model.evaluate(test_X,test_label)

17/17 [==============================] - 2s 143ms/step - loss: 0.7763 - accuracy: 0.7877


[0.7762681841850281, 0.7877094745635986]

In [48]:
model=define_compile_model()

In [49]:
model.load_weights(".mdl_wts.hdf5")

ValueError: Cannot assign value to variable ' dense_21/kernel:0': Shape mismatch.The variable shape (10816, 64), and the assigned value shape (10816, 256) are incompatible.

In [43]:
aug=tf.keras.preprocessing.image.ImageDataGenerator()

In [44]:
aug=tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,fill_mode="nearest",horizontal_flip=True,width_shift_range=0.2,height_shift_range=0.2)